In [43]:
import os
import mysql.connector
from dotenv import load_dotenv
import pandas as pd
import requests
import subprocess
from subprocess import Popen


In [81]:
def conexion():
    load_dotenv(os.path.join('/home/luis/cibercom', '.env'))

    sql_id = os.getenv("SQL_ID")
    sql_pw = os.getenv("SQL_PW")
    #sql_hs = os.getenv("SQL_HOST")
    sql_db = os.getenv("SQL_DB")
    cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host='127.0.0.1', database=sql_db)
    return cnx

def extralias(l):
    cnx = conexion()
    cursor = cnx.cursor()
    
    query  = "select datos_per.coduser, datos_per.nom_apell, "
    query += "datos_red.dir_ip from datos_red, datos_per "
    query += "where datos_per.coduser=datos_red.coduser "
    query += "and datos_red.dir_ip='{}';".format(l)
    cursor.execute(query)
    record = cursor.fetchone()
                
    return record[1][:8]


def learconx(ipserver):
    url ='http://{}/{}'.format(ipserver, 'ips.txt')
    response = requests.get(url) #get data from json file located at specified URL 

    if response.status_code == requests.codes.ok:
        txtResponse = response.text  

        return txtResponse

    else:
        return 'Content was not found.'
    

def valip(ip):
    return len(ip)>0 and ip.find('192.168.10.')==-1 and ip.find('192.168.66.')==-1

def ipsactivos():
    try:
        ipx = learconx('192.168.35.69').split('\n')
        ipx.append('192.168.35.69')
        
    except:
        ipx = []
        
    try:
        ipx.extend(learconx('192.168.35.75').split('\n'))
        ipx.append('192.168.35.75')
    except:
        pass

    ipx = [ix for ix in ipx if valip(ix)]
    ipx = list(set(ipx))
    
    return ipx

def agregacliente():
    clasificacion.append({'path': directorio.replace('/home/luis/cibercom/arbol', 'cibercom'), 'archivo': archivo, 'tipo': tipo, 'referidos': referidos})

def sentinela(ip):
    p = Popen(["/bin/ping", "-c", "2", ip], stdout=subprocess.PIPE, stderr=subprocess.PIPE,
              universal_newlines=True)
    output, errors = p.communicate()
    return output.find("bytes from") > 0

def extrarnodos(path):
    dest = os.walk( path )
    separador = '/'

    nodos = [x for x in dest]
    fcosa = []

    for nodo in nodos:

        if len(nodo[2]) > 0:
            temp = [archivo for archivo in nodo[2]]

            if len(temp) > 0:
                fcosa.append([nodo[0], temp])
    return fcosa


In [44]:
directorios = extrarnodos('/home/luis/cibercom/arbol')

In [48]:
clasificacion = []
activos = ipsactivos

for directorio in directorios:

    tipo = 0 # no activo
    referidos = '#'
    
    with open(os.path.join(directorio, 'ips.txt')) as f:
        f_line = f.readline().replace('\n', '')
        
        while f_line:
            ip = f_line[:11]+f_line[11:].zfill(4)
            f_line = f.readline().replace('\n', '')
            
            if ip in activos or sentinela(ip):
                tipo = 1 # nodo activo
            
                if ip.find('192.168.66.') != -1:
                    # desplegar conex2xxx
                    # y clasificacion.append({'path': director.....
                    pass
                else:
                    # agregar nombre o ip a clasificacion
                    pass
            
            
            
            
            

            clasificacion.append({'path': directorio.replace('/home/luis/cibercom/arbol', 'cibercom'), 'archivo': archivo, 'tipo': tipo, 'referidos': referidos})
            
datos = pd.DataFrame(clasificacion)
datos['grupo'] = 0

Directorio:  /home/luis/cibercom/arbol/mercado 

Directorio:  /home/luis/cibercom/arbol/mercado/blq_17/blq_19/blq_20 

Directorio:  /home/luis/cibercom/arbol/mercado/eduviges 

Directorio:  /home/luis/cibercom/arbol/mercado/eduviges/marron 

Directorio:  /home/luis/cibercom/arbol/mercado/eduviges/marron/cancha_canagua 

Directorio:  /home/luis/cibercom/arbol/mercado/eduviges/marron/cancha_canagua/mar_luz7 

Directorio:  /home/luis/cibercom/arbol/casa 

Directorio:  /home/luis/cibercom/arbol/casa/ksv_bl12 

Directorio:  /home/luis/cibercom/arbol/casa/ksv_bl12/ksb_Y/calle_3 

Directorio:  /home/luis/cibercom/arbol/casa/ksv_bl12/ksb_Y/calle_3/vda_vet 

Directorio:  /home/luis/cibercom/arbol/casa/ksv_bl12/ksb_Y/belks/guarderia 

Directorio:  /home/luis/cibercom/arbol/casa/ksv_bl12/ksb_Y/belks/guarderia/plan1/plan2 

Directorio:  /home/luis/cibercom/arbol/casa/ksv_bl12/ksb_Y/belks/camacho/omaira 

Directorio:  /home/luis/cibercom/arbol/casa/ksv_bl12/ksb_Y/belks/camacho/omaira/benedic 

Dire

In [46]:
import numpy as np

class Arbolito:
    
    def __init__(self, datos):
        
        caminos = datos.path
        
        caminos = list(set(caminos))

        maximo = 0

        temp = []
        for camino in caminos:
            temp.append(camino.split(separador))
            if len(camino.split(separador)) > maximo:
                maximo = len(camino.split(separador))
        caminos = temp.copy()

        alt_caminos = []
        for camino in caminos:
            temp = ''
            for nodo in camino:
                temp += nodo+separador
                alt_caminos.append(temp[:-(len(separador)+len(nodo))]+'**'+ nodo)

        alt_caminos = list(set(alt_caminos))

        for i in range(len(alt_caminos)):
            alt_caminos[i] = alt_caminos[i].split('**')
            alt_caminos[i] = {'direccion': alt_caminos[i][0], 'nombre': alt_caminos[i][1]}

        alt_caminos = np.array(alt_caminos)


        def __buscar_hijos(direccion, nombre):
            hijos = [x['nombre'] for x in alt_caminos if x['direccion']==direccion+nombre+separador]
            return hijos
        
        def __buscar_padre(direccion, nombre):
            dir_list = direccion.split(separador)
            posicion = -2 if len(dir_list)>1 else -1
            padre = dir_list[posicion] # penultimo porque ultimo siempre es vacio por contener separador al final
            return padre

        def __buscar_frutas(direccion, nombre):
            # frutas = list(datos[datos['path']==direccion+nombre]['archivo'].values)
            
            a = datos.loc[datos['path']==direccion+nombre, ['archivo','referidos']]
            frutas = []
            for index, row in a.iterrows():
                i_targets = row['referidos'].split('#')
                targets = [target.split('*') for target in i_targets if len(target)>0]
                source  = row['archivo']
                if len(targets)>0:
                    frutas.extend({'source':source, 'nexo': target[0], 'target': target[1]} for target in targets)
                else:
                    frutas.append({'source':source, 'nexo': 'default','target': nombre})
            return frutas
        
        
        def __numprgs(direccion, nombre):
            a = datos.loc[datos['path']==direccion+nombre]
            prgs   = list(datos[datos['path']==direccion+nombre]['archivo'].values)
            clases = []
            for programa in prgs:
                b = a.loc[a['archivo']==programa, 'grupo'].values[0]
                clases.append([programa, b])
            
            return clases

        def __buscar_nivel(direccion, nombre):
            dir_list = direccion.split(separador)
            return len(dir_list)


        def __ramas():
            
            ramas = []
            
            for camino in alt_caminos:
                parent = __buscar_padre(camino['direccion'], camino['nombre'])
                frutas = __buscar_frutas(camino['direccion'], camino['nombre'])
                progms = __numprgs(camino['direccion'], camino['nombre'])
                hijos  = __buscar_hijos(camino['direccion'], camino['nombre'])
                nivel  = __buscar_nivel(camino['direccion'], camino['nombre'])
                rama = {'parent': parent, 'name': camino['nombre'], 'frutas': frutas, 
                        'children': hijos, 'nivel': str(nivel), 'progms': progms}
                ramas.append(rama)
                
            return ramas    
                
        ramas = __ramas()
        self.ramas = ramas
        
        def __agrega_hijo(abuelo, padre):
            for rama in ramas:

                if rama['name'] == padre and rama['parent'] == abuelo:

                    dat_niv = {'name': str(rama['name']), 'parent': str(rama['parent']),
                               'frutas': rama['frutas'], 'nivel': rama['nivel'], 'progms': rama['progms']}
                    dat_niv['children'] = [__agrega_hijo(rama['name'], children) for children in rama['children']]

                    if dat_niv['children'] == [None] or dat_niv['children'] == [None, None]:
                        dat_niv.pop('children')

                    return dat_niv        
        
        def __crearbol():
            treeData = []
            
            for rama in ramas:
                if int(rama['nivel']) == 1:
                    dat_anid = {'name': rama['name'], 'parent': rama['parent'],
                                'frutas': rama['frutas'], 'nivel': rama['nivel'], 'progms': rama['progms']}
                    dat_anid['children'] = [__agrega_hijo(rama['name'], children) for children in rama['children']]
                    break                
            treeData.append(dat_anid)
            
            return treeData
        
        self.arbol = __crearbol()

In [49]:
a = Arbolito(datos)

treeData = 'var arbol = '+ str(a.arbol) + ';'

treeData = treeData.replace("{'source':", "{source:")
treeData = treeData.replace(" 'target':", " target:")
treeData = treeData.replace(" 'nexo':", " nexo:")
treeData = treeData.replace(".php", "")

with open('arbol.js', 'w') as js_file:
    js_file.write(treeData)

In [80]:
a = ['123', '456', '789']

'456' in a


True